In [13]:
from tkinter import Tk, Label, ttk, StringVar
import numpy as np
from pymongo import MongoClient
import matplotlib.pyplot as plt
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
from matplotlib.figure import Figure

# DB setting
MONGO_HOST = 'localhost'
MONGO_PORT = 27017
MONGO_DB = 'admin'
MONGO_USERNAME = 'root'
MONGO_PASSWORD = 'mongo_password'

def get_database():
    uri = f"mongodb://{MONGO_USERNAME}:{MONGO_PASSWORD}@{MONGO_HOST}:{MONGO_PORT}/"
    client = MongoClient(uri)
    return client[MONGO_DB]

def load_pivot_data_from_mongo():
    db = get_database()
    data = list(db["count_data"].find({}, {"_id": 0}))
    
    dtype = [(key, float) if key not in ["User_ID", "Month"] else (key, int) for key in data[0].keys()]
    data_array = np.array([tuple(d.values()) for d in data], dtype=dtype)
    
    return data_array

# Outlier removal for correlation
def remove_outliers(data, components):
    result = data.copy()
    for comp in components:
        if comp in result.dtype.names:
            values = result[comp]
            q_low = np.quantile(values, 0.15)
            q_high = np.quantile(values, 0.85)
            mask = (values >= q_low) & (values <= q_high)
            result = result[mask]
    return result

def linear_regression(x, y):
    slope, intercept = np.polyfit(x, y, 1)
    return slope, intercept

# Calculate statistics
def calculate_statistics(data, target_components):
    stats_by_month = {}
    months = np.unique(data["Month"])
    for month in months:
        group = data[data["Month"] == month]
        stats_by_month[month] = {
            comp: {
                "mean": round(np.mean(group[comp]), 1) if comp in group.dtype.names else None,
                "median": round(np.median(group[comp]), 1) if comp in group.dtype.names else None,
                "mode": round(np.argmax(np.bincount(group[comp].astype(int))), 1) if comp in group.dtype.names else None
            } for comp in target_components
        }
    # Entire Semester
    stats_by_month["Entire Semester"] = {
        comp: {
            "mean": round(np.mean(data[comp]), 1) if comp in data.dtype.names else None,
            "median": round(np.median(data[comp]), 1) if comp in data.dtype.names else None,
            "mode": round(np.argmax(np.bincount(data[comp].astype(int))), 1) if comp in data.dtype.names else None
        } for comp in target_components
    }
    return stats_by_month

# Load data
pivot_data = load_pivot_data_from_mongo()
target_components = ["Quiz", "Lecture", "Assignment", "Attendance", "Survey"]
correlation_components = ["Assignment", "Quiz", "Lecture", "Book", "Project", "Course"]
stats_by_month = calculate_statistics(pivot_data, target_components)

# Main Menu
def main_menu():
    def open_statistics():
        menu_window.destroy()
        statistics_page()

    def open_correlation():
        menu_window.destroy()
        correlation_page()

    menu_window = Tk()
    menu_window.title("Main Menu")
    menu_window.geometry("400x200")
    Label(menu_window, text="Select an Option:", font=("Arial", 14)).pack(pady=10)
    ttk.Button(menu_window, text="Statistics", command=open_statistics).pack(pady=5)
    ttk.Button(menu_window, text="Correlation", command=open_correlation).pack(pady=5)
    menu_window.mainloop()

# Statistics Page
def statistics_page():
    def go_back():
        stats_window.destroy()
        main_menu()

    def show_statistics():
        for i in stat_tree.get_children():
            stat_tree.delete(i)
        selected = month_var.get()
        data_source = stats_by_month.get("Entire Semester" if selected == "Entire Semester" else int(selected), {})
        for comp, stats in data_source.items():
            stat_tree.insert("", "end", values=(comp, stats["mean"], stats["median"], stats["mode"]))

    stats_window = Tk()
    stats_window.title("Statistics")
    Label(stats_window, text="Select Month:").pack(pady=5)
    month_options = list(stats_by_month.keys())
    month_options_str = [str(m) for m in month_options if m != "Entire Semester"] + ["Entire Semester"]
    month_var = StringVar(value="Entire Semester")
    ttk.Combobox(stats_window, textvariable=month_var, values=month_options_str, state="readonly").pack(pady=5)
    ttk.Button(stats_window, text="Show Statistics", command=show_statistics).pack(pady=5)
    stat_tree = ttk.Treeview(stats_window, columns=("Component", "Mean", "Median", "Mode"), show="headings")
    for col in ["Component", "Mean", "Median", "Mode"]:
        stat_tree.heading(col, text=col)
    stat_tree.pack(pady=10)
    ttk.Button(stats_window, text="Back", command=go_back).pack(pady=5)
    stats_window.mainloop()

# Correlation Page
def correlation_page():
    def go_back():
        corr_window.destroy()
        main_menu()

    corr_window = Tk()
    corr_window.title("Correlation Analysis with Trend Lines")
    fig = Figure(figsize=(8, 6))
    ax = fig.add_subplot(111)

    clean_data = remove_outliers(pivot_data, correlation_components)
    for comp in correlation_components:
        if comp in clean_data.dtype.names:
            x = clean_data["User_ID"]
            y = clean_data[comp]
            ax.scatter(x, y, label=comp, alpha=0.5)
            slope, intercept = linear_regression(x, y)
            ax.plot(x, slope * x + intercept, linestyle='dashed', label=f"{comp} Trend")

    ax.set_xlabel("User_ID")
    ax.set_ylabel("Interactions")
    ax.legend()
    ax.set_title("Correlation Analysis with Trend Lines")
    canvas = FigureCanvasTkAgg(fig, master=corr_window)
    canvas.draw()
    canvas.get_tk_widget().pack()
    ttk.Button(corr_window, text="Back", command=go_back).pack(pady=5)
    corr_window.mainloop()

main_menu()